# New topo DEMs around Willapa Bay

## In collaboration with the Shoalwater Bay Indian Tribe (SBIT)

Under development for the [Cascadia CoPes Hub](https://cascadiacopeshub.org/) project, supported by NSF.


In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import topotools
from clawpack.visclaw import geoplot
from importlib import reload
import geopandas as gpd
import folium
import pandas as pd
import merge_topos

In [ ]:
import find_topo_source

## Load 4 tiles

See the [interactive map](SoWA-Tiles-Map.html) of SW Washington tiles for their locations.

In [ ]:
tile_names = ['n47x00_w124x25', 'n47x00_w124x00', 'n46x75_w124x25', 'n46x75_w124x00']
topos_nc = {}  # to store loaded tiles by key `tile_name`

Use the .nc versions referenced to MHW...

In [ ]:
for tile_name in tile_names:
    tile_urls = find_topo_source.find_tile_url(tile_name, verbose=False)
    url = tile_urls[-2]  # assuming each tile has both a .nc and .tif version
    assert url[-3:] == '.nc', '*** Expected a .nc file!'
    print(f'Loading {tile_name} from \n{url}...')
    topos_nc[tile_name] = topotools.read_netcdf(url)

## Coarsen each tile to 1" and save

In [ ]:
for tile_name in tile_names:
    topo = topos_nc[tile_name]
    topo1s = topo.crop(coarsen=9, align=(-124,46))
    print(f'tile_name: {tile_name},  x[0] = {topo1s.x[0]:.8f}, y[0] = {topo1s.y[0]:.8f}')
    fname = f'{tile_name}_1s.asc'
    topo1s.write(fname, 3, header_style='asc', Z_format='%.3f')
    print(f'Created {fname}')

In [ ]:
ls -lh n4*.asc

## Crop to region near SBIT and merge

In [ ]:
patch_extent = [-124.15, -123.75, 46.6, 46.75]
tileW = 'n46x75_w124x25'
tileE = 'n46x75_w124x00'
topoW_patch_nc = topos_nc[tileW].crop(patch_extent, coarsen=3, align=(-124,46.5), buffer=1)
topoE_patch_nc = topos_nc[tileE].crop(patch_extent, coarsen=3, align=(-124,46.5), buffer=1)

assert abs(topoW_patch_nc.y - topoE_patch_nc.y).max() == 0, '*** y not matching'
y = topoW_patch_nc.y
dx = 1/(3*3600)
x = arange(topoW_patch_nc.x[0], topoE_patch_nc.x[-1]+dx/2, dx)

X,Y = meshgrid(x,y)
Z = zeros(X.shape)
topo_patch_nc = topotools.Topography()
topo_patch_nc.set_xyZ(x,y,Z)
#print('topo_patch_nc extent: ', topo_patch_nc.extent)

topo_patch_nc = merge_topos.overwrite(topo_patch_nc, topoW_patch_nc)
topo_patch_nc = merge_topos.overwrite(topo_patch_nc, topoE_patch_nc)

In [ ]:
fig,ax = subplots(figsize=(10,7))
topo_patch_nc.plot(axes=ax, limits=(-20,20),
                   cb_kwargs={'shrink':0.5, 'extend':'both'})
title('Merged topo at 1/3 arcsecond');
fname = 'WillapaN_13s.png'
savefig(fname)
print(f'Created {fname}')

In [ ]:
fname = 'WillapaN_13s.asc'
topo_patch_nc.write(fname, 3, header_style='asc', Z_format='%.3f')
print(f'Created {fname}')

In [ ]:
ls -lh WillapaN_13s.asc